In [7]:
# 更改数据库格式
import sqlite3
from datetime import datetime

# 连接到SQLite数据库
conn = sqlite3.connect('plan.db')
cursor = conn.cursor()

# 定义替换规则的字典
replacements = {
    'side': {'b': 'buy', 's': 'sell'},
    'type': {'p': 'put', 'c': 'call'},
    'purpose': {'o': 'open', 'c': 'close'}
}

# 更新side、type、purpose列的函数
def update_column(cursor, column, replace_dict):
    cursor.execute(f"SELECT rowid, {column} FROM hedge_option")
    rows = cursor.fetchall()
    
    for rowid, value in rows:
        if value in replace_dict:
            new_value = replace_dict[value]
            cursor.execute(f"UPDATE hedge_option SET {column} = ? WHERE rowid = ?", (new_value, rowid))

# 更新日期列的函数
def update_date_column(cursor):
    cursor.execute("SELECT rowid, date FROM hedge_option")
    rows = cursor.fetchall()
    
    for rowid, date_str in rows:
        if date_str and len(date_str) in (6, 8):  # 处理 MMDDYY 或 MMDDYYYY 格式
            try:
                # 补全年份（MMDDYY -> MMDDYYYY）
                if len(date_str) == 6:
                    # 假设 00-49 为 20XX，50-99 为 19XX
                    year_prefix = "20" if int(date_str[4:6]) < 50 else "19"
                    date_str = date_str + year_prefix
                
                # 转换为标准格式 YYYY-MM-DD
                parsed_date = datetime.strptime(date_str, "%m%d%Y")
                new_date = parsed_date.strftime("%Y-%m-%d")
                cursor.execute("UPDATE hedge_option SET date = ? WHERE rowid = ?", (new_date, rowid))
            except ValueError:
                print(f"日期格式错误，rowid={rowid}，date={date_str}，跳过更新")

# 更新所有指定列
for column, replace_dict in replacements.items():
    update_column(cursor, column, replace_dict)

# 更新日期列
update_date_column(cursor)

# 提交更改并关闭连接
conn.commit()
conn.close()

print("数据库更新完成！")

数据库更新完成！


In [9]:
import sqlite3
from datetime import datetime

# 获取今天的日期，格式 YYYY-MM-DD
today_date = datetime.today().strftime('%Y-%m-%d')  # 例如 "2025-02-21"

# 生成记录函数
def generate_records(symbol, plan_id):
    # 第一行：open 记录
    open_record = (
        symbol,      # symbol
        "buy",       # side
        1,           # quantity
        None,        # type
        "open",      # purpose
        None,        # price
        None,        # strike
        today_date,  # date (动态今天的日期)
        None,        # note
        plan_id      # PlanID
    )
    # 第二行：close 记录
    close_record = (
        symbol,      # symbol
        "sell",      # side
        1,           # quantity
        None,        # type
        "close",     # purpose
        None,        # price
        None,        # strike
        None,        # date
        None,        # note
        plan_id      # PlanID
    )
    return [open_record, close_record]

# 连接数据库
conn = sqlite3.connect('plan.db')
cursor = conn.cursor()

# 查询当前最大的 PlanID 并生成新的唯一值
cursor.execute("SELECT MAX(PlanID) FROM hedge_option")
max_plan_id = cursor.fetchone()[0]  # 获取最大值
if max_plan_id is None:
    new_plan_id = 1  # 如果表为空，从1开始
else:
    new_plan_id = max_plan_id + 1  # 递增1，确保唯一

# 指定 symbol
symbol = "UNH"  # 可改为任意值，如 "AAPL"

# 生成并插入记录
records = generate_records(symbol, new_plan_id)
cursor.executemany("""
INSERT INTO "hedge_option" ("symbol", "side", "quality", "type", "purpose", "price", "strike", "date", "note", "PlanID")
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""", records)

# 获取刚插入的两行记录的 rowid
cursor.execute("SELECT last_insert_rowid()")
first_rowid = cursor.fetchone()[0] - 1  # 第一行 rowid
second_rowid = first_rowid + 1          # 第二行 rowid

# 打印刚插入的两行
cursor.execute("SELECT * FROM hedge_option WHERE rowid IN (?, ?)", (first_rowid, second_rowid))
print("新生成的记录：")
for row in cursor.fetchall():
    print(row)

# 提交更改并关闭连接
conn.commit()
conn.close()

print(f"已为 symbol={symbol} 生成并插入两行交易记录，PlanID={new_plan_id}！")

新生成的记录：
('UNH', 'buy', 1, None, 'open', None, None, '2025-02-21', None, 7)
('UNH', 'sell', 1, None, 'close', None, None, None, None, 7)
已为 symbol=UNH 生成并插入两行交易记录，PlanID=7！
